In [ ]:
import pandas as pd
import numpy as np
import os, re
import joblib
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense, Input
from sklearn.preprocessing import MinMaxScaler

# 경로 설정
VGA_PATH = '/content/VGA_Total'
if not os.path.exists(VGA_PATH):
    os.makedirs(VGA_PATH, exist_ok=True)
    print("⚠️ /content/VGA_Total 폴더에 CSV 파일을 업로드한 후 다시 실행하세요.")
else:
    # 데이터 로드 및 전처리 (904개 관측치 기반)
    all_files = sorted([f for f in os.listdir(VGA_PATH) if f.endswith('.csv')])
    all_data = []
    
    for f in all_files:
        df_tmp = pd.read_csv(os.path.join(VGA_PATH, f), encoding='cp949') # 혹은 utf-8
        # RTX 4060 등 특정 타겟 추출 로직 (예시: RTX 4060)
        target_rows = df_tmp[df_tmp['Name'].str.contains('4060', na=False)]
        date_cols = [c for c in df_tmp.columns if re.match(r'\d{4}-\d{2}-\d{2}', c)]
        for col in date_cols:
            p = pd.to_numeric(target_rows[col].astype(str).str.replace(',', '').str.extract('(\d+)')[0], errors='coerce')
            if not p.dropna().empty:
                all_data.append({'Date': col, 'Price': p.mean()})

    df = pd.DataFrame(all_data).groupby('Date')['Price'].mean().reset_index().sort_values('Date')
    df['Price_Smooth'] = df['Price'].rolling(window=3, min_periods=1).mean()

    # 스케일링 및 모델 빌드
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(df[['Price_Smooth']])
    
    SEQ_LENGTH = 30
    X, y = [], []
    for i in range(len(scaled_data)-SEQ_LENGTH):
        X.append(scaled_data[i:i+SEQ_LENGTH])
        y.append(scaled_data[i+SEQ_LENGTH])
    
    X, y = np.array(X), np.array(y)

    # 사용자 지정 LSTM 모델 구조 [user's provided code]
    model = Sequential([
        Input(shape=(SEQ_LENGTH, 1)),
        LSTM(128, return_sequences=True),
        Dropout(0.2),
        LSTM(64),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='huber')
    model.fit(X, y, epochs=50, batch_size=16, verbose=0) # 학습 진행

    # 모델 저장
    model.save('/content/vga_model.h5')
    joblib.dump(scaler, '/content/vga_scaler.pkl')
    print("✅ 모델 학습 및 vga_model.h5 생성 완료")